In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchvision.models.vgg import VGG16_Weights
import matplotlib.pyplot as plt
import pathlib
import os
import pandas as pd
import random
from PIL import Image
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight
import cv2
from tqdm import tqdm
import time
from os import listdir
from os.path import isfile, join

In [15]:
class CustomTrainDataset(Dataset):
    def __init__(self, annotations_file, image_directory, transform=None, label_encoder=None , split='train', test_size=0.2, random_state=None):
        self.labels = pd.read_csv(annotations_file)
        self.image_directory = image_directory
        self.transform = transform
        self.label_encoder = label_encoder
        train_labels, val_labels = train_test_split(self.labels, test_size=test_size, random_state=random_state)
        if split == 'train':
            self.labels = train_labels
        elif split == 'val':
            self.labels = val_labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
      image_path = os.path.join(self.image_directory, self.labels.iloc[idx]['File Name'])
      image = Image.open(image_path).convert("RGB")
      if self.transform:
          image = self.transform(image)
      label = self.labels.iloc[idx]['Category']
      if self.label_encoder:
          label = self.label_encoder.transform([label])[0]
      return image, label

In [16]:
annotations_file = '/content/drive/MyDrive/train.csv'
image_directory = '/content/drive/MyDrive/minichallenge_train/train_preprocessed'

## Use of label encoder for classification: Idea from https://discuss.pytorch.org/t/how-to-encode-labels-for-classification-on-custom-dataset/142396
labels_df = pd.read_csv(annotations_file)
labels = labels_df['Category'].tolist()
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

##Different data augmentations: Idea from https://colab.research.google.com/drive/109vu3F1LTzD1gdVV6cho9fKGx7lzbFll
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    transforms.RandomAffine(0, translate=(0.1, 0.1)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

val_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

train_dataset = CustomTrainDataset(annotations_file=annotations_file, image_directory= image_directory, transform=train_transforms, label_encoder=label_encoder, split='train', test_size=0.2, random_state=42)
val_dataset = CustomTrainDataset(annotations_file=annotations_file, image_directory= image_directory, transform=val_transforms, label_encoder=label_encoder, split='val', test_size=0.2, random_state=42)

train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True, num_workers=12, pin_memory=True, prefetch_factor=36)
val_loader = DataLoader(val_dataset, batch_size=256, shuffle=False, num_workers=12, pin_memory=True, prefetch_factor=36)

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.vgg16(weights=VGG16_Weights.DEFAULT)

## Fitting a pre-trained classifier with custom number of classes, idea from https://github.com/pytorch/vision/issues/3547
num_classes = len(label_encoder.classes_)
num_features = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_features, num_classes)
model = model.to(device)

## Basing the Cross Entropy Loss on balanced class weights, idea from https://stackoverflow.com/questions/69783897/compute-class-weight-function-issue-in-sklearn-library-when-used-in-keras-cl
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(encoded_labels), y=encoded_labels)
class_weights_tensor = torch.tensor(class_weights, dtype=torch.float)
criterion = nn.CrossEntropyLoss(weight=class_weights_tensor.to(device))

## Using ReduceLROnPlateau, idea from https://discuss.pytorch.org/t/reduce-lr-on-plateau-based-on-training-loss-or-validation/183344
optimizer = torch.optim.AdamW([{'params': model.features.parameters(), 'lr': 0.0, 'frozen': True}, {'params': model.classifier.parameters(), 'lr': 0.001, 'frozen': False}], weight_decay=1e-4)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5)

In [ ]:
best_val_loss_so_far = 100000
patience = 50
epoch_with_no_progress = 0
num_epochs = 200
initial_lr = 0.001
last_lr = None

unfreezing_schedule = {5: 4, 10: 8, 15: 12, 20: 'all'}
for param in model.features.parameters():
    param.requires_grad = False

for epoch in range(num_epochs):
    # Unfreezing VGG16 layers gradually: Idea from https://discuss.huggingface.co/t/gradual-layer-freezing/3381/4
    if epoch in unfreezing_schedule.keys():
        unfreezing_layers = unfreezing_schedule[epoch]
        if unfreezing_layers == 'all':
            for param in model.features.parameters():
                param.requires_grad = True
            for param_group in optimizer.param_groups:
                if param_group.get('frozen', False):
                    param_group['lr'] = initial_lr / 10
                    param_group.pop('frozen', None)
        else:
            layer_list = list(model.features.children())
            for layer in layer_list[-unfreezing_layers:]:
                for param in layer.parameters():
                    param.requires_grad = True
                for param_group in optimizer.param_groups:
                    if 'frozen' in param_group and param_group['frozen'] == True:
                      param_group['lr'] = initial_lr / 10
                      param_group.pop('frozen', None)
    # Training loop: Idea of tqdm as a progress bar from https://adamoudad.github.io/posts/progress_bar_with_tqdm/
    model.train()
    running_loss = 0.0
    train_loader = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} Training")

    for batch_idx, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        train_loader.set_description(f"Epoch {epoch+1}/{num_epochs}, Train Loss: {running_loss / (batch_idx + 1):.4f}")

    train_loss = running_loss / len(train_loader)
    print(f'Epoch: {epoch+1}/{num_epochs} , Train Loss: {train_loss:.6f}')

    # Validation loop: Idea of validation loop structure from: https://www.geeksforgeeks.org/training-neural-networks-with-validation-using-pytorch/
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        val_loader = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} Validation")
        for batch_idx, (images, labels) in enumerate(val_loader):
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            val_loader.set_description(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss / (batch_idx + 1):.4f}")

    val_loss /= len(val_loader)
    val_acc = 100. * correct / total
    print(f'Epoch: {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.6f}, Validation Acc: {val_acc:.2f}%')
    scheduler.step(val_loss)

    # Early stopping: Idea from https://stackoverflow.com/questions/71998978/early-stopping-in-pytorch
    if val_loss < best_val_loss_so_far:
        print(f'Got a new model that gives the best validation loss so far, saving to Drive!')
        torch.save(model.state_dict(), "drive/MyDrive/best_model_so_far.pth")
        best_val_loss_so_far = val_loss
        epoch_with_no_progress = 0
    else:
        epoch_with_no_progress += 1
        if epoch_with_no_progress >= patience:
            print("Early stopping triggered due to no progress over validation loss!")
            break

Epoch 1/200, Train Loss: 4.3195:  60%|█████▉    | 130/218 [19:26<01:03,  1.39it/s]

In [ ]:
class CustomTestDataset(Dataset):
    def __init__(self, image_directory, transform=None):
        self.image_directory = image_directory
        self.transform = transform
        self.image_files = [f for f in os.listdir(image_directory) if os.path.isfile(os.path.join(image_directory, f))]

    def __getitem__(self, idx):
        image_path = os.path.join(self.image_directory, self.image_files[idx])
        image = Image.open(image_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, self.image_files[idx]

test_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_dataset = CustomTestDataset(image_directory='drive/MyDrive/test_preprocessed', transform=test_transform)

test_loader = DataLoaderval_loader = DataLoader(
    test_dataset,
    batch_size=256,
    shuffle=False,
    num_workers=12,
)

In [ ]:
predictions = []
file_names_list = []

model.eval()

with torch.no_grad():
    for images, file_names in test_loader:
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        predictions.extend(predicted.cpu().numpy())
        file_names_list.extend(file_names)

In [ ]:
predicted_labels = label_encoder.inverse_transform(predictions)
indices = [int(f.split('.')[0]) for f in file_names_list]

results_df = pd.DataFrame({
    'Id': indices,
    'Category': predicted_labels
})

results_df = results_df.sort_values(by='Id').reset_index(drop=True)
results_df.to_csv('drive/MyDrive/test_predictions.csv', index=False)